In [1]:
import os
from dotenv import load_dotenv, dotenv_values 
from langchain.chat_models import ChatOpenAI

load_dotenv()
api_key = os.getenv("MY_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", openai_api_key=api_key)

C:\Users\gssabijp\AppData\Local\Temp\ipykernel_30452\4008285106.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", openai_api_key=api_key)


In [2]:
template_string = """Create a json file consisting of 20 restaurants in new york,
let the keys be name of the restaurant, price for two people and a 3-sentence review
explaining how people felt about the food, make sure the reviews are random if one review says food was good let one say that the food was bad.
Give me the output as comma separated values with the header restaurant, price for two(in $), review which can be easily converted to a dataframe,
give only the output without the csv text and triple quotes
"""

In [3]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
customer_messages = prompt_template.format_messages()
response = llm(customer_messages)
a = response.content
a

C:\Users\gssabijp\AppData\Local\Temp\ipykernel_30452\3698132983.py:5: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(customer_messages)


'restaurant,price for two(in $),review\n"Joe\'s Pizza",30,"The pizza at Joe\'s is a classic New York experience. Many patrons rave about the perfect crust and fresh toppings. However, some found it a bit too greasy for their taste."\n"Le Bernardin",300,"Le Bernardin offers an exquisite dining experience with its seafood dishes. Diners often leave in awe of the flavors and presentation. A few, however, felt the prices were not justified for the portion sizes."\n"Shake Shack",25,"Shake Shack is loved for its juicy burgers and crinkle-cut fries. Customers appreciate the casual vibe and quick service. Some, though, think the hype doesn\'t match the taste."\n"Peter Luger Steak House",150,"Peter Luger\'s is renowned for its steaks, and many say it\'s the best they\'ve ever had. The atmosphere is classic and inviting. Yet, a few diners were disappointed by the service they received."\n"Momofuku Noodle Bar",40,"Momofuku\'s ramen is a favorite among locals, with rich broth and perfectly cooked 

In [4]:
import pandas as pd
from io import StringIO
# Step 2: Convert to DataFrame
csv_data = StringIO(a)

# Step 2: Read the CSV data into a DataFrame
df = pd.read_csv(csv_data)

# Step 3: Display the DataFrame
print(df)

                 restaurant  price for two(in $)  \
0               Joe's Pizza                   30   
1              Le Bernardin                  300   
2               Shake Shack                   25   
3   Peter Luger Steak House                  150   
4       Momofuku Noodle Bar                   40   
5       Eleven Madison Park                  350   
6           Los Tacos No. 1                   20   
7                   Carbone                  200   
8     Russ & Daughters Cafe                   50   
9                 Blue Hill                  150   
10              Smorgasburg                   40   
11              Shake Shack                   25   
12         Keens Steakhouse                  200   
13                Bar Pitti                   70   
14                  Tacombi                   30   
15              Momofuku Ko                  200   
16              ABC Kitchen                  150   
17    Frankies 457 Spuntino                   80   
18          

In [5]:
all_sentiments = []
for review in df["review"]:
    prompt = f'''
        Classify the following review 
        as having either a positive or
        negative sentiment. State your answer
        as a single word, either "positive" or
        "negative":

        {review}
        '''
    prompt_template = ChatPromptTemplate.from_template(prompt)
    customer_messages = prompt_template.format_messages()
    response = llm(customer_messages)
    all_sentiments.append(response.content)
    #all_sentiments.append(all_sentiments)
all_sentiments

['positive',
 'positive',
 'mixed',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'mixed',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

In [6]:
df = pd.DataFrame(all_sentiments, columns=['review'])

# Count occurrences
count_df = df['review'].value_counts().reset_index()
count_df.columns = ['review', 'Count']
count_df

,review,Count
0,positive,16
1,mixed,2
2,negative,1
